<a href="https://colab.research.google.com/github/KU81-ComSci-ProjectEnd-GameCard/main/blob/master/Predictor/data/json_unzip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# load data
**https://drive.google.com/drive/folders/1c7MwTdLxnPgvmPbBEfNWa45YAUU53H0l** make sure to load data form this before processing.  

raw_data file so large to push to github

In [10]:
import re
def clear_SearingBlowPlus(lst):
    return [re.sub(r'Searing Blow\+\d+', 'Searing Blow', item) for item in lst]

In [11]:
def upgrade_lst(lst):
  new_lst = list()
  for item in lst:
    if(item == 'Searing Blow'):
      new_lst.append('Searing Blow')
    else:
      new_lst.append(item+'+1')
  return new_lst

In [12]:
def get_ALL_CARD():
  IC_ATTACK_CARDS = ["Strike_R", "Bash", "Anger", "Body Slam", "Clash", "Cleave", "Clothesline", "Headbutt", "Heavy Blade", "Iron Wave", "Perfected Strike", "Pommel Strike", "Sword Boomerang", "Thunderclap", "Twin Strike", "Wild Strike", "Blood for Blood", "Carnage", "Dropkick", "Hemokinesis", "Pummel", "Rampage", "Reckless Charge", "Searing Blow", "Sever Soul", "Uppercut", "Whirlwind", "Bludgeon", "Feed", "Fiend Fire", "Immolate", "Reaper"]
  IC_SKILL_CARDS = ["Defend_R", "Armaments", "Flex", "Havoc", "Shrug It Off", "True Grit", "Warcry", "Battle Trance", "Bloodletting", "Burning Pact", "Disarm", "Dual Wield", "Entrench", "Flame Barrier", "Ghostly Armor", "Infernal Blade", "Intimidate", "Power Through", "Rage", "Second Wind", "Seeing Red", "Sentinel", "Shockwave", "Spot Weakness", "Double Tap", "Exhume", "Impervious", "Limit Break", "Offering"]
  IC_POWER_CARDS = ["Combust", "Dark Embrace", "Evolve", "Feel No Pain", "Fire Breathing", "Inflame", "Metallicize", "Rupture", "Barricade", "Berserk", "Brutality", "Corruption", "Demon Form", "Juggernaut"]
  IRONCLAD_CARDS = IC_ATTACK_CARDS + IC_SKILL_CARDS + IC_POWER_CARDS

  COLORLESS_CARDS = ["Dramatic Entrance", "Flash of Steel", "Mind Blast", "Swift Strike", "HandOfGreed", "Bite", "Expunger", "Ritual Dagger", "Shiv", "Smite", "Through Violence", "Bandage Up", "Blind", "Dark Shackles", "Deep Breath", "Discovery", "Enlightenment", "Finesse", "Forethought", "Good Instincts", "Impatience", "Jack Of All Trades", "Madness", "Panacea", "PanicButton", "Purity", "Trip", "Apotheosis", "Chrysalis", "Master of Strategy", "Metamorphosis", "Secret Technique", "Secret Weapon", "The Bomb", "Thinking Ahead", "Transmutation", "Violence", "Apparition", "Beta", "Insight", "J.A.X.", "Miracle", "Safety", "Magnetism", "Mayhem", "Panache", "Sadistic Nature"]

  CURSE_CARDS = ["AscendersBane", "Clumsy", "Curse of the Bell", "Decay", "Doubt", "Injury", "Necronomicurse", "Normality", "Pain", "Parasite", "Pride", "Regret", "Shame", "Writhe"]
  return IRONCLAD_CARDS + COLORLESS_CARDS + CURSE_CARDS + upgrade_lst(IRONCLAD_CARDS) + upgrade_lst(COLORLESS_CARDS)

In [13]:
import pandas as pd
def get_ic_run(json_data):
  df = pd.DataFrame(json_data)
  runInfo = df['event'].apply(pd.Series)
  runInfo['master_deck'] = runInfo['master_deck'].apply(clear_SearingBlowPlus)
  ## รับแค่ run ของ ironclad
  ALL_CARD = get_ALL_CARD()
  ic_run = runInfo[runInfo['master_deck'].apply(lambda x: all(word in ALL_CARD for word in x))]
  ## เกณคือ ต้องผ่านชั้น 50 แล้ว
  return ic_run.loc[ic_run.floor_reached >= 50]

In [14]:
import os
import gzip
import json

def get_quality_run(directory, run_num,start=0, end=None):
    quality_run = pd.DataFrame()
    try:
        counter = 0
        for file_name in os.listdir(directory):
            file_path = os.path.join(directory, file_name)
            if os.path.isfile(file_path) and file_name.endswith('.json.gz'):
                if counter >= start and (end is None or counter < end):
                    with gzip.open(file_path, 'rt', encoding='utf-8') as gz_file:
                        run_info = json.load(gz_file)
                        ic_run = get_ic_run(run_info)
                        quality_run = pd.concat([quality_run, ic_run], axis=0, join='outer')
                        quality_run.drop_duplicates(subset=['play_id'])
                        print(counter, '   :     ',len(quality_run))
                        if(len(quality_run) >= run_num):
                            break
            counter += 1
    except Exception as e:
        print(f"Error: {e}")
    return quality_run
                        

In [6]:
raw_data = 'Monthly_2020_11'

In [7]:
quality_run = get_quality_run(raw_data, 2000)

0    :      50
1    :      100
2    :      152
3    :      204
4    :      267
5    :      325
6    :      362
7    :      410
8    :      462
9    :      511
10    :      549
11    :      600
12    :      636
13    :      684
14    :      734
15    :      784
16    :      848
17    :      907
18    :      975
19    :      1029
20    :      1089
21    :      1159
22    :      1213
23    :      1286
24    :      1338
25    :      1406
26    :      1462
27    :      1510
28    :      1558
29    :      1615
30    :      1685
31    :      1753
32    :      1811
33    :      1864
34    :      1920
35    :      1967
36    :      2041


In [8]:
quality_run = quality_run.reset_index()

In [16]:
import os
file_path = os.path.join('processed_data', 'quality_run2k.json')
quality_run.to_json(file_path)

In [15]:
quality_run.loc[quality_run.ascension_level >= 10]

,index,gold_per_floor,floor_reached,playtime,items_purged,score,play_id,local_time,is_ascension_mode,campfire_choices,...,relics_obtained,event_choices,is_beta,boss_relics,items_purged_floors,is_endless,potions_floor_spawned,killed_by,ascension_level,special_seed
7,191,"[111, 111, 125, 125, 16, 42, 42, 74, 98, 124, ...",55,3489,[],1265,fa1fe12c-ceac-405f-bd33-09fb290ae3e0,20201101015911,True,"[{'data': 'Immolate', 'floor': 7, 'key': 'SMIT...",...,"[{'floor': 4, 'key': 'FossilizedHelix'}, {'flo...","[{'cards_removed': ['Defend_R'], 'damage_heale...",False,"[{'not_picked': ['SlaversCollar', 'Runic Cube'...",[],False,"[1, 3, 8, 13, 16, 20, 21, 22, 35, 41, 45, 48]",The Heart,10,0.0
13,344,"[117, 135, 155, 165, 176, 176, 207, 207, 207, ...",52,4814,[Pain],1812,b31329ae-cbaa-4b6c-82ee-ce4edac81d47,20201101080028,True,"[{'data': 'Infernal Blade', 'floor': 6.0, 'key...",...,"[{'floor': 7.0, 'key': 'Darkstone Periapt'}, {...","[{'damage_healed': 0.0, 'max_hp_gain': 0.0, 'm...",False,"[{'not_picked': ['Busted Crown', 'Calling Bell...",[30],False,"[2, 3, 12, 14, 16, 20, 21, 22, 24, 33, 38, 40,...",NaN,20,0.0
14,417,"[112, 112, 112, 131, 56, 85, 85, 120, 120, 153...",56,3517,"[Strike_R, Defend_R]",2238,765522c4-0c87-4518-b6af-974d4a055c57,20201101010114,True,"[{'data': 'Flex', 'floor': 7.0, 'key': 'SMITH'...",...,"[{'floor': 6.0, 'key': 'Red Skull'}, {'floor':...","[{'cards_removed': ['Strike_R'], 'damage_heale...",False,"[{'not_picked': ['Pandora's Box', 'Runic Dome'...","[5, 22]",False,"[1, 13, 16, 23, 24, 27, 31, 33, 35, 37, 38]",The Heart,20,0.0
15,472,"[110, 110, 185, 65, 65, 65, 79, 97, 124, 124, ...",50,9023,[Defend_R],754,2a5df035-584c-4965-845d-523fdddf2f50,20201101010144,True,"[{'data': 'Armaments', 'floor': 6, 'key': 'SMI...",...,"[{'floor': 9, 'key': 'Akabeko'}, {'floor': 14,...","[{'damage_healed': 0, 'gold_gain': 0, 'player_...",False,"[{'not_picked': ['Calling Bell', 'Ectoplasm'],...",[4],False,"[7, 13, 14, 18, 25, 28, 30, 35, 38, 41]",Awakened One,13,NaN
20,568,"[111, 111, 111, 2, 16, 16, 52, 65, 65, 65, 98,...",50,3761,[Defend_R],1153,bfa19dd1-f201-4844-a0f7-439f13983712,20201101100246,True,"[{'floor': 6, 'key': 'REST'}, {'data': 'Inflam...",...,"[{'floor': 7, 'key': 'Mummified Hand'}, {'floo...","[{'damage_healed': 0, 'gold_gain': 0, 'player_...",False,"[{'not_picked': ['Runic Dome', 'Ectoplasm'], '...",[28],False,"[1, 5, 7, 8, 18, 31, 37, 39, 45]",Awakened One,20,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020,992,"[10, 10, 28, 46, 57, 57, 68, 68, 68, 96, 96, 1...",51,4804,"[Defend_R, Clumsy]",1122,05087c4a-4f7a-4886-9049-f85afb8f533d,20201101150705,True,"[{'data': 'Cleave', 'floor': 6.0, 'key': 'SMIT...",...,"[{'floor': 8.0, 'key': 'Eternal Feather'}, {'f...","[{'damage_healed': 0.0, 'max_hp_gain': 0.0, 'm...",False,"[{'not_picked': ['Mark of Pain', 'SacredBark']...","[21, 28]",False,"[4, 10, 16, 18, 23, 27, 31, 33, 35, 42, 46, 48]",NaN,13,0.0
2022,1008,"[110, 124, 48, 48, 198, 198, 225, 244, 244, 16...",51,3315,[Doubt],1281,66211c79-ea9a-449a-b079-dbeb3e9d212f,20201031230715,True,"[{'data': 'Bash', 'floor': 6.0, 'key': 'SMITH'...",...,"[{'floor': 7.0, 'key': 'Bottled Lightning'}, {...","[{'damage_healed': 0.0, 'max_hp_gain': 0.0, 'm...",False,"[{'not_picked': ['Black Blood', 'Busted Crown'...",[10],False,"[1, 7, 12, 30, 39]",NaN,18,0.0
2027,1082,"[111, 121, 61, 24, 41, 70, 83, 83, 83, 108, 10...",50,1328,[],921,4b3c5c5a-a42b-452e-92a9-5736367ea8f1,20201101010800,True,"[{'floor': 8, 'key': 'REST'}, {'floor': 11, 'k...",...,"[{'floor': 6, 'key': 'Gambling Chip'}, {'floor...","[{'damage_healed': 0, 'gold_gain': 0, 'player_...",False,"[{'not_picked': ['Ectoplasm', 'Coffee Dripper'...",[],False,"[6, 7, 10, 14, 19, 28, 30, 35, 48]",Donu and Deca,15,NaN
2030,1091,"[164, 174, 26, 101, 111, 136, 136, 166, 192, 2...",56,2698,[],1881,6d195545-f598-4797-a55d-7ff14f03272f,20201101070759,True,"[{'floor': 7.0, 'key': 'REST'}, {'data': 'Perf...",...,"[{'floor': 6.0, 'key': 'Oddly Smoot